<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">OpenAI Node SDK Tracing</h1>

Let's see how to get started with using the OpenAI Node SDK to trace your LLM calls using Deno. 

> Note: that this example requires the OPENAI_API_KEY environment variable to be set and assumes you are running the Phoenix server on localhost:6006.

In [1]:
import {
  NodeTracerProvider,
  SimpleSpanProcessor,
} from "npm:@opentelemetry/sdk-trace-node";
import { Resource } from "npm:@opentelemetry/resources";
import { OTLPTraceExporter } from "npm:@opentelemetry/exporter-trace-otlp-proto";
import { SEMRESATTRS_PROJECT_NAME } from "npm:@arizeai/openinference-semantic-conventions";
import { diag, DiagConsoleLogger, DiagLogLevel } from "npm:@opentelemetry/api";

// For troubleshooting, set the log level to DiagLogLevel.DEBUG
diag.setLogger(new DiagConsoleLogger(), DiagLogLevel.INFO);

const provider = new NodeTracerProvider({
  resource: new Resource({
    [SEMRESATTRS_PROJECT_NAME]: "deno-openai",
  }),
});

provider.addSpanProcessor(
  new SimpleSpanProcessor(
    new OTLPTraceExporter({
      url: "http://localhost:6006/v1/traces",
    }),
  ),
);

provider.register();

console.log("👀 OpenInference initialized");

👀 OpenInference initialized


In [2]:
import OpenAI from 'npm:openai';
import { OpenAIInstrumentation } from "npm:@arizeai/openinference-instrumentation-openai";

const oaiInstrumentor = new OpenAIInstrumentation();
oaiInstrumentor.manuallyInstrument(OpenAI);

In [4]:
const client = new OpenAI({
  apiKey: process.env['OPENAI_API_KEY'], // This is the default and can be omitted
});

async function main() {
 try {
  const tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
          },
          "required": ["location"],
        },
      }
    }
];
const messages =  [{ role: 'user', content: 'Whats the weather like in seattle today?' }];
  const chatCompletion = await client.chat.completions.create({
    messages,
    tools,
    model: 'gpt-3.5-turbo',
  });
  const responseMessage = chatCompletion.choices[0].message;
  console.dir(chatCompletion.choices[0].message);
  
  messages.push(responseMessage)
  messages.push({role: "tool", content: "{\"location\": \"Seattle, WA\", \"unit\": \"fahrenheit\", \"temperature\": 70}", tool_call_id: responseMessage.tool_calls[0].id})
  const toolCallCompletion = await client.chat.completions.create({
    messages,
    tools,
    model: 'gpt-3.5-turbo',
  })
  
  console.dir(toolCallCompletion.choices[0].message);
 } catch (e) {
   console.error(e);
 }
}

await main();

{
  role: "assistant",
  content: null,
  tool_calls: [
    {
      id: "call_VRb3npx3FjU5DpXrfvLpMruO",
      type: "function",
      function: {
        name: "get_current_weather",
        arguments: '{"location":"Seattle, WA"}'
      }
    }
  ],
  refusal: null
}
{
  role: "assistant",
  content: "The current weather in Seattle, WA is 70°F.",
  refusal: null
}


In [7]:
;

async function main() {
 try {
  const tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
          },
          "required": ["location"],
        },
      }
    }
];
const messages =  [{ role: 'user', content: 'Whats the weather like in seattle today?' }];
  const chatCompletion = await client.chat.completions.create({
    messages,
    tools,
    model: 'gpt-3.5-turbo',
  });
  const responseMessage = chatCompletion.choices[0].message;
  console.dir(chatCompletion.choices[0].message);
  
  // This will fail because we don't have the tool message preceeded by a non tool message with tool calls
//   messages.push(responseMessage)
  messages.push({role: "tool", content: "{\"location\": \"Seattle, WA\", \"unit\": \"fahrenheit\", \"temperature\": 70}", tool_call_id: responseMessage.tool_calls[0].id})
  const toolCallCompletion = await client.chat.completions.create({
    messages,
    tools,
    model: 'gpt-3.5-turbo',
  })
  
  console.dir(toolCallCompletion.choices[0].message);
 } catch (e) {
   console.error(e);
 }
}

await main();

{
  role: "assistant",
  content: null,
  tool_calls: [
    {
      id: "call_H9QOPTfREver2M34p7uRnnwb",
      type: "function",
      function: {
        name: "get_current_weather",
        arguments: '{"location":"Seattle","unit":"celsius"}'
      }
    }
  ],
  refusal: null
}


Error: 400 Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    at Function.generate (file:///Users/parkerstafford/Library/Caches/deno/npm/registry.npmjs.org/openai/4.67.1/error.mjs:41:20)
    at OpenAI.makeStatusError (file:///Users/parkerstafford/Library/Caches/deno/npm/registry.npmjs.org/openai/4.67.1/core.mjs:284:25)
    at OpenAI.makeRequest (file:///Users/parkerstafford/Library/Caches/deno/npm/registry.npmjs.org/openai/4.67.1/core.mjs:328:30)
    at eventLoopTick (ext:core/01_core.js:175:7)
    at async main (<anonymous>:51:32)
    at async <anonymous>:61:1 {
  status: 400,
  headers: {
    "access-control-expose-headers": "X-Request-ID",
    "alt-svc": 'h3=":443"; ma=86400',
    "cf-cache-status": "DYNAMIC",
    "cf-ray": "8d4a6f7d3de7a3bf-SEA",
    "content-length": "233",
    "content-type": "application/json",
    date: "Fri, 18 Oct 2024 18:01:45 GMT",
    "openai-organization": "arize-ai-ewa7w1",
    "openai-processin